# SSD Object Detection Demo

This notebook will walk you step by step through the process of using a pre-trained model to detect objects in a video. 
* You may first go over the `object_detection_tutorial` notebook in the same folder as this notebook is residing in.
* The code in this notebook is from [lazyprogrammer SSD](https://github.com/lazyprogrammer/machine_learning_examples/blob/master/cnn_class2/ssd.py)

# Imports

In [19]:
import numpy as np
import os
import sys
from datetime import datetime
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
    raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

print(tf.__version__)

1.8.0


In [11]:
import imageio

In [10]:
# This is needed to display the images.
%matplotlib inline

In [3]:
# # change this to wherever you cloned the tensorflow models repo
# # which I assume you've already downloaded from:
# # https://github.com/tensorflow/models
# RESEARCH_PATH = '../../tf-models/research'
# MODELS_PATH = '../../tf-models/research/object_detection'
# sys.path.append(RESEARCH_PATH)
# sys.path.append(MODELS_PATH)

In [13]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

In [14]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Load a (frozen) Tensorflow model into memory.
* Assume that you have already downloaded the model
* check `object_detection_tutorial` notebook for detail of downloading the SSD model

In [15]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [16]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [17]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [22]:
# do some object detection
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        # instead of looping through test images, we'll now loop
        # through our video!

        # get the videos from:
        # https://lazyprogrammer.me/cnn_class2_videos.zip
        # and put them into the same folder as this file

        # open the video
        
        input_video = 'catdog'
#         input_video = 'safari'
#         input_video = 'traffic'

        # read video from videos folder
        video_reader = imageio.get_reader('videos/%s.mp4' % input_video)
    
        # put annotated video to videos_out folder
        video_writer = imageio.get_writer('videos_out/%s_annotated.mp4' % input_video, fps=10)

        # loop through and process each frame
        t0 = datetime.now()
        n_frames = 0
        for frame in video_reader:
            # rename for convenience
            image_np = frame
            n_frames += 1

            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)

            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})

            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
      
            # instead of plotting image, we write the frame to video
            video_writer.append_data(image_np)

        fps = n_frames / (datetime.now() - t0).total_seconds()
        print("Frames processed: %s, Speed: %s fps" % (n_frames, fps))

        # clean up
        video_writer.close()

Frames processed: 50, Speed: 4.820915522711623 fps


# Have fun

* You can use your own webcam and see how SSD can be used to annotate you and objects in your environment
* You need to install openCV which is somewhat non-trivial